In [33]:
import numpy as np
import pandas as pd


raw_train_data = pd.read_csv('covid_train.csv')



{1, 2}

In [ ]:
def preprocessing(data):
  data['age_range'] = pd.cut(x=data['age'], bins=[0,10,20,30,40,50,60,70,80,90,100,120], labels=['0s','10s','20s','30s','40s','50s','60s','70s','80s','90s','100s']) 
  return data.drop(['entry_date', 'date_symptoms','age'], axis=1)


In [ ]:
def NB(raw_data):
  data = preprocessing(raw_data)

  prior_p = np.zeros(2)
  prior_p[0] = sum(data['date_died']=='9999-99-99')/len(data['date_died'])
  prior_p[1] = 1 - prior_p[0]

  x = data.drop(['date_died'], axis=1)
  cd = {}
  for col in range(x.shape[0]):
    feature_cd = np.zeros(2,len(set(x.iloc[0,col]))
    for n in len(set(x.iloc[0,col])):
      feature_cd[0,n] = 
      feature_cd[1,n] = 



